# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,66.60,88,75,21.99,PT,1685710753
1,1,keren,15.7778,38.4581,86.68,31,82,9.26,ER,1685710754
2,2,adamstown,-25.0660,-130.1015,75.09,87,97,22.77,PN,1685710754
3,3,bethel,41.3712,-73.4140,79.45,68,0,0.00,US,1685710754
4,4,motygino,58.1831,94.7592,70.61,58,88,4.70,RU,1685710754


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
map_plot_city = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", frame_width=700, frame_height=500, 
                                           size="Humidity", scale=0.5, color="City")

# Display the map
map_plot_city

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_condition = ((city_data_df["Max Temp"] > 0) & (city_data_df["Max Temp"] < 60))
wind_condition = (city_data_df["Wind Speed"] < 4.5)
cloud_condition = (city_data_df["Cloudiness"] == 0)

city_data_filter = city_data_df.loc[(temp_condition & wind_condition & cloud_condition ), :]

# Drop any rows with null values
city_data_final = city_data_filter.dropna()

# Display sample data
city_data_final

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
97,97,anadyr,64.7500,177.4833,34.30,59,0,4.47,RU,1685710777
150,150,winslow,35.0242,-110.6974,57.38,41,0,3.44,US,1685710788
224,224,uyuni,-20.4597,-66.8250,37.67,30,0,1.83,BO,1685710822
254,254,portland,45.5234,-122.6762,49.06,88,0,1.99,US,1685710761
268,268,abra pampa,-22.7205,-65.6970,50.83,12,0,1.50,AR,1685710850
302,302,newman,37.3138,-121.0208,54.28,85,0,3.00,US,1685710858
351,351,alice springs,-23.7000,133.8833,54.95,41,0,0.00,AU,1685710879
379,379,llallagua,-18.4167,-66.6333,47.93,16,0,2.75,BO,1685710891
427,427,patterson,37.4716,-121.1297,55.98,85,0,3.20,US,1685710683
518,518,salmon arm,50.6998,-119.3024,54.43,76,0,3.44,CA,1685710924


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_final.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
97,anadyr,RU,64.7500,177.4833,59,
150,winslow,US,35.0242,-110.6974,41,
224,uyuni,BO,-20.4597,-66.8250,30,
254,portland,US,45.5234,-122.6762,88,
268,abra pampa,AR,-22.7205,-65.6970,12,
302,newman,US,37.3138,-121.0208,85,
351,alice springs,AU,-23.7000,133.8833,41,
379,llallagua,BO,-18.4167,-66.6333,16,
427,patterson,US,37.4716,-121.1297,85,
518,salmon arm,CA,50.6998,-119.3024,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 5
apiKey = geoapify_key
params = {
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    params["categories"] = f'accommodation.hotel'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
        
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
anadyr - nearest hotel: Гостевой дом
winslow - nearest hotel: La Posada Hotel
uyuni - nearest hotel: Hostel Jerian
portland - nearest hotel: the Hoxton
abra pampa - nearest hotel: Residencia El Norte
newman - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
llallagua - nearest hotel: No hotel found
patterson - nearest hotel: Hampton Inn & Suites
salmon arm - nearest hotel: SureStay Plus Hotel by Best Western Salmon Arm


,City,Country,Lat,Lng,Humidity,Hotel Name
97,anadyr,RU,64.7500,177.4833,59,Гостевой дом
150,winslow,US,35.0242,-110.6974,41,La Posada Hotel
224,uyuni,BO,-20.4597,-66.8250,30,Hostel Jerian
254,portland,US,45.5234,-122.6762,88,the Hoxton
268,abra pampa,AR,-22.7205,-65.6970,12,Residencia El Norte
302,newman,US,37.3138,-121.0208,85,No hotel found
351,alice springs,AU,-23.7000,133.8833,41,Aurora Alice Springs
379,llallagua,BO,-18.4167,-66.6333,16,No hotel found
427,patterson,US,37.4716,-121.1297,85,Hampton Inn & Suites
518,salmon arm,CA,50.6998,-119.3024,76,SureStay Plus Hotel by Best Western Salmon Arm


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
#%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", frame_width=700, frame_height=500, size="Humidity", 
                                        scale=0.5, color="City", hover_cols = ["Hotel Name","Country"])

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)